In [1]:
import numpy as np
import tensorflow as tf
print(tf.__version__)

1.1.0


## TensorFlow config (optional)
TensorFlow는 디폴트 설정에 의해 GPU 메모리를 모두 점유하기 때문에 다수가 접속하는 서버에서는 아래와 같은 설정을 통해 이를 방지하도록 합니다.

In [2]:
sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))

## TensorFlow 세션

In [3]:
sess = tf.Session(config=sess_config)

a = tf.ones((2, 2))
b = tf.reduce_sum(a, axis=1)
c = tf.reshape(a, (1, 4))

print(a)
print(sess.run(a))
print(sess.run(b))
print(sess.run(c))

sess.close()

Tensor("ones:0", shape=(2, 2), dtype=float32)
[[ 1.  1.]
 [ 1.  1.]]
[ 2.  2.]
[[ 1.  1.  1.  1.]]


In [4]:
a = tf.constant([[5, 6]])
b = tf.constant([[5], [6]])
c = a*b

with tf.Session(config=sess_config) as sess:
    print(sess.run(a))
    print(sess.run(b))
    print(sess.run(c))

[[5 6]]
[[5]
 [6]]
[[25 30]
 [30 36]]


## TensorFlow 변수

In [5]:
w1 = tf.Variable(tf.random_normal([5, 2], stddev=0.1))
w2 = tf.Variable([[3, 2, 3], [1, 2, 3]])

with tf.Session(config=sess_config) as sess:
    sess.run(tf.global_variables_initializer())
    w1_fetch, w2_fetch = sess.run([w1, w2])
    print(w1_fetch)
    print(w2_fetch)

[[-0.08993217 -0.13364786]
 [ 0.15204987  0.08158098]
 [-0.00338036 -0.09407447]
 [-0.07901672  0.06409153]
 [-0.08955514 -0.00680354]]
[[3 2 3]
 [1 2 3]]


In [6]:
state = tf.Variable(0, name="counter")
new_value = tf.add(state, tf.constant(1))
update = tf.assign(state, new_value)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("inital: ", sess.run(state))
    for i in range(3):
        sess.run(update)
        print("update {}: ".format(i), sess.run(state))

inital:  0
update 0:  1
update 1:  2
update 2:  3


## TensorFlow Placeholder

In [7]:
a = tf.placeholder(tf.float32, name="a")
b = tf.placeholder(tf.float32, name="b")

print(a)
print(b)

add = a+b
mul = a*b

with tf.Session() as sess:
    print(sess.run(add, feed_dict={a: 2, b: 3}))
    print(sess.run(mul, feed_dict={a: 4, b: 5}))

Tensor("a:0", dtype=float32)
Tensor("b:0", dtype=float32)
5.0
20.0


In [8]:
matrix1 = tf.placeholder(tf.float32, [2, 2])
matrix2 = tf.placeholder(tf.float32, [2, 2])
product = tf.matmul(matrix1, matrix2)

with tf.Session() as sess:
    mv1 = np.array([[3., 3.], [2., 2.]])
    mv2 = np.array([[2., 1.], [2., 1.]])
    result = sess.run(product, feed_dict={matrix1: mv1, matrix2: mv2})
    print(mv1)
    print(mv2)
    print(result)

[[ 3.  3.]
 [ 2.  2.]]
[[ 2.  1.]
 [ 2.  1.]]
[[ 12.   6.]
 [  8.   4.]]


## 퍼셉트론으로 AND, OR 문제 풀기

In [9]:
and_x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
and_y = np.array([0, 0, 0, 1])
or_x  = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
or_y  = np.array([0, 1, 1, 1])

learning_rate = 0.1

inputs = tf.placeholder(tf.float32, [None, 2])
label  = tf.placeholder(tf.int64, [None])
weight = tf.Variable(tf.random_normal([2, 2]))
bias   = tf.Variable(tf.zeros([2]))

logit = tf.matmul(inputs, weight) + bias
pred_op = tf.nn.softmax(logit)

loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=label, logits=logit))
opt = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_op)

with tf.Session(config=sess_config) as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(500):
        _, loss = sess.run([opt, loss_op], feed_dict={inputs:and_x, label:and_y})
        if (step+1)%100 == 0:
            pred = sess.run(pred_op, feed_dict={inputs:and_x, label:and_y})
            print("step", step+1, "loss:", loss, "gt:", and_y, "predict:", np.argmax(pred, axis=1))

step 100 loss: 0.360255 gt: [0 0 0 1] predict: [0 0 0 1]
step 200 loss: 0.251539 gt: [0 0 0 1] predict: [0 0 0 1]
step 300 loss: 0.197987 gt: [0 0 0 1] predict: [0 0 0 1]
step 400 loss: 0.164111 gt: [0 0 0 1] predict: [0 0 0 1]
step 500 loss: 0.140213 gt: [0 0 0 1] predict: [0 0 0 1]


## 퍼셉트론으로 XOR 문제 풀기

In [10]:
xor_x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
xor_y = np.array([0, 1, 1, 0])

learning_rate = 0.1

inputs = tf.placeholder(tf.float32, [None, 2])
label  = tf.placeholder(tf.int64, [None])
weight = tf.Variable(tf.random_normal([2, 2]))
bias   = tf.Variable(tf.zeros([2]))

logit = tf.matmul(inputs, weight) + bias
pred_op = tf.nn.softmax(logit)

loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=label, logits=logit))
opt = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_op)

with tf.Session(config=sess_config) as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10000):
        _, loss = sess.run([opt, loss_op], feed_dict={inputs:xor_x, label:xor_y})
        if (step+1)%1000 == 0:
            pred = sess.run(pred_op, feed_dict={inputs:xor_x, label:xor_y})
            print("step", step+1, "loss:", loss, "gt:", xor_y, "predict:", np.argmax(pred, axis=1))

step 1000 loss: 0.693147 gt: [0 1 1 0] predict: [1 1 1 0]
step 2000 loss: 0.693147 gt: [0 1 1 0] predict: [1 1 0 0]
step 3000 loss: 0.693147 gt: [0 1 1 0] predict: [1 1 0 0]
step 4000 loss: 0.693147 gt: [0 1 1 0] predict: [1 1 0 0]
step 5000 loss: 0.693147 gt: [0 1 1 0] predict: [1 1 0 0]
step 6000 loss: 0.693147 gt: [0 1 1 0] predict: [1 1 0 0]
step 7000 loss: 0.693147 gt: [0 1 1 0] predict: [1 1 0 0]
step 8000 loss: 0.693147 gt: [0 1 1 0] predict: [1 1 0 0]
step 9000 loss: 0.693147 gt: [0 1 1 0] predict: [1 1 0 0]
step 10000 loss: 0.693147 gt: [0 1 1 0] predict: [1 1 0 0]


## 멀티 레이어 퍼셉트론으로 XOR 문제 풀기

In [11]:
xor_x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
xor_y = np.array([1, 0, 0, 1])

learning_rate = 0.1

inputs = tf.placeholder(tf.float32, [None, 2])
label  = tf.placeholder(tf.int64, [None])

W1 = tf.Variable(tf.random_normal([2, 2]))
b1 = tf.Variable(tf.zeros([2]))
W2 = tf.Variable(tf.random_normal([2, 2]))
b2 = tf.Variable(tf.zeros([2]))

layer1 = tf.matmul(inputs, W1) + b1
layer1 = tf.sigmoid(layer1)
logit = tf.matmul(layer1, W2) + b2
pred_op = tf.nn.softmax(logit)

loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=label, logits=logit))
opt = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_op)

with tf.Session(config=sess_config) as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10000):
        _, loss = sess.run([opt, loss_op], feed_dict={inputs:xor_x, label:xor_y})
        if (step+1)%1000 == 0:
            pred = sess.run(pred_op, feed_dict={inputs:xor_x, label:xor_y})
            print("step", step+1, "loss:", loss, "gt:", xor_y, "predict:", np.argmax(pred, axis=1))

step 1000 loss: 0.688865 gt: [1 0 0 1] predict: [1 1 0 0]
step 2000 loss: 0.57285 gt: [1 0 0 1] predict: [1 0 0 0]
step 3000 loss: 0.246368 gt: [1 0 0 1] predict: [1 0 0 1]
step 4000 loss: 0.0634705 gt: [1 0 0 1] predict: [1 0 0 1]
step 5000 loss: 0.0314783 gt: [1 0 0 1] predict: [1 0 0 1]
step 6000 loss: 0.0203247 gt: [1 0 0 1] predict: [1 0 0 1]
step 7000 loss: 0.0148462 gt: [1 0 0 1] predict: [1 0 0 1]
step 8000 loss: 0.0116326 gt: [1 0 0 1] predict: [1 0 0 1]
step 9000 loss: 0.00953384 gt: [1 0 0 1] predict: [1 0 0 1]
step 10000 loss: 0.00806128 gt: [1 0 0 1] predict: [1 0 0 1]
